<html>
<body>

<h1 style="background-color:pink;""font-size:300%;">BREAST CANCER GENE DATA</h1>
</body>
</html>


![][cancer]

[cancer]: https://www.uicc.org/sites/main/files/styles/uicc_news_main_image/public/thumbnails/image/BCAM2016_FA.jpg?itok=zimiEGKS.png "Picture of cancer awareness"

<html>
<body>

<h1 style="background-color:pink;">About Breast Cancer:</h1>
<p style="font-family:verdana;">Breast cancer is cancer that forms in the cells of the breasts.After skin cancer, breast cancer is the most common cancer diagnosed in women in the United States. Breast cancer can occur in both men and women, but it's far more common in women.Substantial support for breast cancer awareness and research funding has helped created advances in the diagnosis and treatment of breast cancer. Breast cancer survival rates have increased, and the number of deaths associated with this disease is steadily declining, largely due to factors such as earlier detection, a new personalized approach to treatment and a better understanding of the disease.</p>
</body>
</html>


<html>
<body>

<h1 style="background-color:pink;">Import necessary packages</h1>
</body>
</html>


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

import plotly.express as px
import plotly.figure_factory as ff
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.simplefilter('ignore')

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from xgboost import plot_tree, plot_importance

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

<html>
<body>

<h1 style="background-color:pink;">Reading the data:</h1>
</body>
</html>


In [ ]:
data = pd.read_csv("../input/breastcancerdataset/BRCA.csv")
data.head()

<html>
<body>

<h1 style="background-color:pink;">Checking for null values:</h1>
</body>
</html>


In [ ]:
data.isnull().sum()

In [ ]:
data = data.dropna()

In [ ]:
data.shape

In [ ]:
data['Patient_Status'].value_counts()

<html>
<body>

<h1 style="background-color:pink;">Basic Analysis of Features:</h1>
</body>
</html>


In [ ]:
#fixing custom colors based on the aesthetic of this notebook
custom_colors = ["#B61D46","#C47DA9","#A77165","#CB9794","#824A59"]
customPalette = sns.set_palette(sns.color_palette(custom_colors))

<html>
<body>

<h1 style="background-color:pink;">Numerical Features:</h1>
</body>
</html>


In [ ]:
numerical_features = [data['Protein1'], data['Protein2'], data['Protein3'], data['Protein4']]
group_labels = ['Protein1', 'Protein2', 'Protein3', 'Protein4']
custom_color = ["#9B174A","#56B1AC","#BDC0CA","#CB9794"]

fig = ff.create_distplot(numerical_features, group_labels, curve_type='normal',histnorm ='probability density', bin_size=[.1, .3,.4,.5]
    ,show_rug=True,colors=custom_color)

fig.show()

In [ ]:
fig = px.scatter_matrix(data,
    dimensions= ['Age','Protein1','Protein2','Protein3','Protein4'],
    color = 'Patient_Status',color_continuous_scale= px.colors.sequential.Burg,template='ggplot2')
fig.update_layout(
    title='Relationship between Numerical Features',
    width=900,
    height=900,
)

fig.show()

<html>
<body>

<h1 style="background-color:pink;">Categorical Features:</h1>
</body>
</html>

In [ ]:
le = LabelEncoder()
data['Patient_Status']=le.fit_transform(data['Patient_Status'].astype(str))

<html>
<body>
<h1 style="background-color:pink;">Parallel Categories:</h1>
<p style="font-family:verdana;">The parallel categories diagram (also known as parallel sets or alluvial diagram) is a visualization of multi-dimensional categorical data sets. Each variable in the data set is represented by a column of rectangles, where each rectangle corresponds to a discrete value taken on by that variable. The relative heights of the rectangles reflect the relative frequency of occurrence of the corresponding value.Combinations of category rectangles across dimensions are connected by ribbons, where the height of the ribbon corresponds to the relative frequency of occurrence of the combination of categories in the data set.</p>

</body>
</html>

In [ ]:
fig = px.parallel_categories(data, dimensions=['Gender','Tumour_Stage','ER status','PR status','HER2 status'],
                             color = 'Patient_Status',
                 color_continuous_scale=px.colors.sequential.Burg,template='plotly_dark')
fig.show()

<html>
<body>
<h1 style="background-color:pink;">Model:</h1>
</body>
</html>

[Code Contributor🎗](https://www.kaggle.com/dharineeshk)

In [ ]:
def model_function(name,kernel=''):
    if name =='Logisitic Regression':
        model = LogisticRegression()
    elif name =='Decision Tree':
        model = DecisionTreeClassifier()
    elif name =='Random Forest':
        model = RandomForestClassifier()
    elif name == 'Naive Bayes':
        model = GaussianNB()
    elif name == "Stochastic Gradient Descent":
        model = SGDClassifier()
    elif name == 'KNN':
        model = KNeighborsClassifier()
        
    elif name == "Cross Gradient Booster":
        model = XGBClassifier(verbosity = 0)
    elif name == "Cross Gradient Booster (Random Forest)":
        model = XGBRFClassifier(verbosity = 0)
    elif name =='SVM':
        if kernel == 'Linear':
            model = SVC(kernel='linear')
        if kernel == 'RBF':
            model = SVC(kernel='rbf')
        if kernel == 'Sigmoid':
            model = SVC(kernel='sigmoid')
            
    # Prediction Scores
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    
    return score

In [ ]:
le = LabelEncoder()
objList=['Gender','Tumour_Stage','ER status','PR status','HER2 status']
for feat in objList:
    data[feat] = le.fit_transform(data[feat].astype(str))


In [ ]:
data=data.drop(['Histology','Surgery_type','Date_of_Surgery','Date_of_Last_Visit'],axis=1)

In [ ]:
data.head()

In [ ]:
models = ['Logistic regression','Decision Tree','Random Forest','Naive Bayes','Stochastic Gradient Descent','KNN','SVM-Linear',
             'SVM-RBF','SVM-Sigmoid','XGB','XGB(RF)']
accuracy_data = pd.DataFrame(models)
data = data.iloc[0:, 1:] 
X = data.iloc[:,:-1]
y = data['Patient_Status']
    
cols = X.columns
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X)
X = pd.DataFrame(np_scaled, columns = cols)
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
scores = []
scores.append(model_function('Logisitic Regression'))
scores.append(model_function('Decision Tree'))  
scores.append(model_function('Random Forest'))
scores.append(model_function('Naive Bayes'))
scores.append(model_function('Stochastic Gradient Descent'))
scores.append(model_function('KNN'))
scores.append(model_function('SVM','Linear'))
scores.append(model_function('SVM','RBF'))
scores.append(model_function('SVM','Sigmoid'))
scores.append(model_function('Cross Gradient Booster'))
scores.append(model_function('Cross Gradient Booster (Random Forest)'))

scores = pd.Series(scores)

print('Calculated Scores!')

In [ ]:
scores.index = models
accuracy = scores.to_frame()

In [ ]:
accuracy

<<html>
<body>
<h1 style="background-color:pink;">Accuracy : 82% </h1>
</body>
</html>